In [7]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, chisquare
from sklearn.cluster import KMeans

## Shared functions

In [8]:
def perc(value, decimals=1):
    return round(100*value, decimals)


def compute_perc(df, stat_columns=[], remde_delay=None, recycle_remde_delay=True, remde_dead_threshold=None):
    max_cluster = max(df.cluster)

    n_total = df.shape[0]
    if recycle_remde_delay==False:
        n_total = df[(df.remdesivir != True) | (df.remde_delay <= remde_delay)].shape[0]
    
    rows = []
    for cluster in range(1, max_cluster+1):
        row = []
        row.append(cluster) # cluster number
        patients = df[df.cluster == cluster]
        
        remde = patients[patients.remdesivir == True] 
        no_remde = patients[patients.remdesivir != True] 
        if remde_delay:
            remde = patients[(patients.remdesivir == True) & (patients.remde_delay <= remde_delay)]
            if recycle_remde_delay:
                no_remde = patients[(patients.remdesivir != True) | (patients.remde_delay > remde_delay)]
                
        if remde_dead_threshold!=None:
            remde_dead_patients = remde[remde.x_dead_after_remde_days<remde_dead_threshold]
            if not remde_dead_patients.empty:
                print(f"The following patients will move from remde to no remde:")
                display(remde_dead_patients)
                remde = remde[remde.x_dead_after_remde_days>=remde_dead_threshold]
                no_remde = pd.concat(no_remde, remde_dead_patients)
                
        total_remde = remde.shape[0]
        total_no_remde = no_remde.shape[0]
        n_cluster = total_remde + total_no_remde
        row.append(n_cluster) # n of cluster
        row.append(perc(n_cluster/n_total)) # % of total n
        row.append(total_remde)
        row.append(total_no_remde)
        
        perc_death_remde = f"{sum(remde.dead_60d)} ({perc(sum(remde.dead_60d)/total_remde)}%)" if total_remde else None
        perc_death_no_remde = f"{sum(no_remde.dead_60d)} ({perc(sum(no_remde.dead_60d)/total_no_remde)}%)" if total_no_remde else None
        total_death = sum(remde.dead_60d) + sum(no_remde.dead_60d)
        total = total_remde + total_no_remde
        perc_total_death = f"{total_death} ({perc(total_death /(total))}%)" if total else None
        row.append(perc_death_remde)
        row.append(perc_death_no_remde)
        row.append(perc_total_death)
        
        stat, pvalue = None, None
        if len(remde.dead_60d)>1 and len(no_remde.dead_60d)>1:
            stat, pvalue = ttest_ind(list(remde.dead_60d), list(no_remde.dead_60d))
            
        row.append(pvalue)
        row.append(stat)
        
        for stat_column in stat_columns:
            remde_val = f"({round(remde[stat_column].min(),1)}/{round(remde[stat_column].mean(),1)}/{round(remde[stat_column].max(),1)})"
            noremde_val = f"({round(no_remde[stat_column].min(),1)}/{round(no_remde[stat_column].mean(),1)}/{round(no_remde[stat_column].max(),1)})"
            row.append(f"{remde_val} / {noremde_val}")
            
        rows.append(row)
    
    stats = pd.DataFrame(rows, columns=['cluster', 'n', '%', 'n Remde', 'n No Remde', '% Death Remde', '% Death No Remde', '% Death', 'pvalue', 'stat'] + stat_columns)
    stats.set_index('cluster', inplace=True)
    
    return stats


MAX_CT = 41.7
MAX_DDXCOV = 22
MAX_LINFOCITOS = 4.2
def normalize(df_orig):
    df = df_orig.copy()
    df.linfocitos = df_orig.linfocitos/(MAX_LINFOCITOS)
    df.ct = df_orig.ct/MAX_CT
    df.ddxcov = df_orig.ddxcov/MAX_DDXCOV
    return df
    

# Same as before with another k_centers format
def sci_add_cluster_column(df, k_centers, column_name='cluster', add_to=None):
    if isinstance(add_to, None.__class__):
        add_to = df.copy()
    values = df[['ct', 'ddxcov', 'linfocitos']]
    
    # Iterate through the entries
    clusters = []
    for i in range(0,len(values)):
        distances = []
        # Compute distance to 6 centers
        pn = values.iloc[i]
        for j in range(0, k_centers.shape[0]):
            center = k_centers[j]
            distance = np.linalg.norm(pn - center)
            distances.append(distance)

        clusters.append(distances.index(min(distances))+1)

    add_to.insert(0, column_name, clusters)
    return add_to


def from01toTrueFalse(df, columns):
    df2 = df.copy()
    for column in columns:
        df2[column] = False
        mask = df[column] == 1
        df2.loc[mask,column] = True
    return df2


def shows(df, col, cluster):
    df1 = df.loc[df.cluster == cluster, col]
    a,b,c = df1.quantile([0.25,0.5,0.75])
    res = f"({round(a,1)}/{round(b,1)}/{round(c,1)})"
    return res
    

# Load data

## Load Hospital Clinic

In [9]:
hc_rename = {
    'Ct': 'ct',
    'DeltaDDXCOVSimptomes': 'ddxcov',
    'Linfocitos_ing': 'linfocitos',
    'MORT_60d': 'dead_60d',
    'DeltaRemdesivirSimptomes': 'remde_delay',
    'Remdesivir': 'remdesivir',
    'Edad': 'age',
}

hc_orig = pd.read_csv('hc.csv').rename(columns=hc_rename)
hc_orig['src'] = 'hc'
hc_orig.set_index('id', inplace=True, verify_integrity=True)

In [20]:
hc_norm = normalize(hc_orig)
hc = hc_orig

## Load Terrassa

In [11]:
mt_rename = {
    'Limfòcits': 'linfocitos',
    'remdesivir_date': 'remde_date',
}

mt_orig = pd.read_csv('terrassa.csv')
mt_orig = mt_orig.rename(columns=mt_rename)
mt_orig['src'] = 'mt'
mt_orig_nona = mt_orig.dropna(subset=['sex', 'age', 'dead_60d', 'covid_date', 'ct', 'ddxcov', 'linfocitos']).copy()
mt_orig_nona['covid_date'] = pd.to_datetime(mt_orig_nona.covid_date, format='%Y-%m-%d %H:%M:%S')
mt_orig.shape, mt_orig_nona.shape

mtpremarch = mt_orig_nona[mt_orig_nona.covid_date < pd.Timestamp(2021,3,1)]
mtpremarch.shape

mt = mtpremarch
mt_norm = normalize(mt)
mt.shape

(447, 72)

## Load Valencia

In [12]:
valencia_rename = {
    'CT': 'ct',
    'DDX': 'ddxcov',
    'LINFOCITOS': 'linfocitos',
    'MORT_60d': 'dead_60d',
    'Remdesivir': 'remdesivir',
    'NUM': 'pid'
}

valencia_orig = pd.read_csv('lafe.csv').rename(columns=valencia_rename)
valencia_orig['src'] = 'val'
valencia_orig.drop(columns=['Unnamed: 0'], inplace=True)
valencia_orig = from01toTrueFalse(valencia_orig, ['UCI', 'remdesivir', 'dead_60d'])
valencia_orig.linfocitos = valencia_orig.linfocitos/1000
valencia_orig['remde_delay'] = 1
valencia_orig['death_date']= None 
valencia_orig['remde_date'] = None
valencia_orig['covid_date'] = None

In [13]:
valencia_norm = normalize(valencia_orig)
valencia = valencia_orig
valencia = valencia.drop(columns='cluster')
valencia.shape

(455, 12)

## Join Valencia and Terrassa cohorts

In [14]:
mt_val = pd.concat([mt[valencia.columns], valencia])
mt_val_norm = normalize(mt_val)

# KMeans 

In [15]:
km_data = hc_norm[['ct', 'ddxcov', 'linfocitos', ]]
kmeans_hc = KMeans(n_clusters=5, random_state=0, n_init=10).fit(km_data)

In [16]:
hc = sci_add_cluster_column(hc_norm, kmeans_hc.cluster_centers_, add_to=hc)
hc.to_csv('hc_w_cluster.csv')
compute_perc(hc, stat_columns=['linfocitos', 'ct', 'age'], remde_delay=6)

,n,%,n Remde,n No Remde,% Death Remde,% Death No Remde,% Death,pvalue,stat,linfocitos,ct,age
cluster,,,,,,,,,,,,
1,100,8.6,16,84,0 (0.0%),2 (2.4%),2 (2.0%),0.537735,-0.618417,(1.2/1.8/3.8) / (1.2/1.9/4.2),(11.7/23.8/32.1) / (13.1/26.4/37.0),(49/64.2/86) / (25/59.2/92)
2,273,23.5,7,266,0 (0.0%),30 (11.3%),30 (11.0%),0.348133,-0.939847,(0.2/0.7/1.3) / (0.2/0.8/1.7),(19.0/23.5/26.4) / (4.3/23.7/29.7),(40/52.0/75) / (23/65.3/94)
3,183,15.8,0,183,None,15 (8.2%),15 (8.2%),NaN,NaN,(nan/nan/nan) / (0.3/0.9/3.1),(nan/nan/nan) / (20.5/30.9/39.0),(nan/nan/nan) / (20/62.4/96)
4,318,27.4,34,284,1 (2.9%),32 (11.3%),33 (10.4%),0.133291,-1.505126,(0.2/0.8/1.3) / (0.2/0.8/1.6),(25.6/29.5/38.0) / (25.3/31.2/41.7),(28/58.9/91) / (16/62.9/99)
5,286,24.7,76,210,8 (10.5%),77 (36.7%),85 (29.7%),0.000015,-4.400507,(0.1/0.7/1.6) / (0.1/0.7/1.7),(10.8/20.2/26.6) / (10.2/20.2/28.0),(24/66.4/89) / (26/73.2/96)


## Stats on validation

In [17]:
mt_val = sci_add_cluster_column(mt_val_norm, kmeans_hc.cluster_centers_, add_to=mt_val)
compute_perc(mt_val, stat_columns=['linfocitos', 'ct'], remde_delay=6)

,n,%,n Remde,n No Remde,% Death Remde,% Death No Remde,% Death,pvalue,stat,linfocitos,ct
cluster,,,,,,,,,,,
1,167,18.5,15,152,0 (0.0%),11 (7.2%),11 (6.6%),0.283825,-1.075267,(1.2/2.2/7.0) / (1.3/2.5/88.1),(19.0/26.3/30.0) / (12.0/25.2/38.0)
2,292,32.4,31,261,1 (3.2%),20 (7.7%),21 (7.2%),0.367695,-0.902208,(0.4/0.9/1.3) / (0.2/0.9/2.1),(11.0/20.5/28.0) / (7.0/21.0/29.0)
3,121,13.4,1,120,0 (0.0%),9 (7.5%),9 (7.4%),NaN,NaN,(1.0/1.0/1.0) / (0.4/1.2/3.5),(31.0/31.0/31.0) / (14.0/28.8/39.0)
4,156,17.3,29,127,4 (13.8%),16 (12.6%),20 (12.8%),0.863246,0.172532,(0.3/0.9/1.4) / (0.3/0.9/1.7),(26.0/30.2/35.0) / (26.0/31.0/39.0)
5,166,18.4,45,121,5 (11.1%),31 (25.6%),36 (21.7%),0.044061,-2.029138,(0.2/0.8/1.4) / (0.2/0.8/1.6),(9.0/19.0/26.0) / (10.0/19.2/27.0)


## Cluster statistics

In [18]:

src = hc
rows = []
COLUMNS =  ['ct', 'linfocitos', 'ddxcov']
for i in range(1,max(src.cluster)+1):
    rows.append([i] + [shows(src, col=col, cluster=i) for col in COLUMNS])
df= pd.DataFrame(rows, columns=['cluster'] + COLUMNS).set_index('cluster')
df


,ct,linfocitos,ddxcov
cluster,,,
1,(23.0/26.1/29.9),(1.5/1.7/2.0),(3.0/5.0/7.0)
2,(21.7/24.0/26.2),(0.6/0.8/1.0),(7.0/8.0/9.0)
3,(28.5/31.0/33.7),(0.6/0.8/1.1),(10.0/11.0/13.0)
4,(29.0/30.8/33.0),(0.6/0.8/1.0),(4.0/5.0/7.0)
5,(17.1/20.5/23.0),(0.5/0.7/0.9),(1.0/3.0/4.0)


In [19]:

src = mt_val
rows = []
COLUMNS =  ['ct', 'linfocitos', 'ddxcov']
for i in range(1,max(src.cluster)+1):
    rows.append([i] + [shows(src, col=col, cluster=i) for col in COLUMNS])
df= pd.DataFrame(rows, columns=['cluster'] + COLUMNS).set_index('cluster')
df


,ct,linfocitos,ddxcov
cluster,,,
1,(22.0/25.0/28.5),(1.6/1.8/2.2),(3.5/6.0/7.0)
2,(18.0/21.0/25.0),(0.7/0.9/1.1),(7.0/8.0/9.0)
3,(26.0/30.0/32.0),(0.8/1.1/1.5),(10.0/12.0/14.0)
4,(29.0/31.0/33.0),(0.7/0.9/1.2),(3.0/5.0/7.0)
5,(16.0/19.0/22.0),(0.6/0.8/1.0),(2.0/3.0/4.0)
